# Trends Accumulated in time range
- This Notebook `obtains and saves` within `"reports/trends_accumulated"` the `Trends accumulated` in a given `date range`.

##### What exactly does the script do?
- Receives a `DATE_FROM` and `DATE_TO` range.
- Obtains all the `Trends` within the `DB` in that `time range`.
- Unifies into the same DataFrame, adding the `"volume"` column for equal Trends.
- Concatenates the unique values non-empty within `"domainContext"`.
- Save the result in a `.CSV` inside `"reports/trends_accumulated"`.

---

##### You can modify these parameters.
- `WITH_SAVE:` If `True`, saves it to `"reports/trends_accumulated"`, `False` does not save it.
- `DATE_FROM:` Initial date of the analysis (inclusive).
- `DATE_TO:` End date of the analysis (inclusive).

---

- *Note_1: If there are no Trends on any of the dates in the interval, you will be notified with a text here.*
- *Note_2: If there is at least 1 day with Trends in the interval, it will do the accumulation.*
- *Note_3: If Trends in days are missing at the beginning or end, it will limit the interval where there are no gaps at the edges.*

In [ ]:
from datetime import datetime

from scraping_kit import load_db_and_bots

db_tw, bots = load_db_and_bots()

WITH_SAVE = True
DATE_FROM = datetime(year=2023, month=12, day=28)
DATE_TO = datetime(year=2024, month=1, day=21)
df_accumulated = db_tw.get_trends_df_accumulated(date_from=DATE_FROM, date_to=DATE_TO, with_save=WITH_SAVE)
df_accumulated[:50]

In [ ]:
from typing import List, Literal
from scraping_kit.db.db_twitter import DBTwitter
from scraping_kit.db.models.topics import Topic, TopicClasses
def texts_from_trend(trend_name: str) -> List[str]:
    return [t["text"] for t in db_tw.coll.search.find_one({"query": trend_name})["timeline"]]
def topic_from_trend(trend_name: str) -> Topic:
    return Topic(**db_tw.coll.topics.find_one({"query": trend_name}))

def choice_i(old_val: Literal[1,2,3,4] = None) -> int:
    if old_val == 1 or old_val == 3:
        return 0
    elif old_val == 2 or old_val == 4:
        return 1
    else:
        raise Exception("aksdna")

In [ ]:
def print_topic(topic: Topic) -> None:
    print("~"*10)
    display(f"topics_1: {topic.topics_1.labels[:2]}")
    display(f"topicS_2: {topic.topics_2.labels[:2]}")
    print("~"*10)

def change_ij_topic(i: int, j: int, topic_classes: TopicClasses) -> None:
    topic_classes.labels[i], topic_classes.labels[j] = topic_classes.labels[j], topic_classes.labels[i]

def update_topic(topic: Topic, new_val: str = None, old_val: Literal[1,2,3,4] = None, silence=True) -> None:
    display(topic.query)
    print_topic(topic)
    if new_val is not None and old_val in range(1, 5):
        i = choice_i(old_val)
        if old_val in [1,2]:
            j = topic.topics_1.labels.index(new_val)
            change_ij_topic(i, j, topic.topics_1)
        elif old_val in [3,4]:
            j = topic.topics_2.labels.index(new_val)
            change_ij_topic(i, j, topic.topics_2)
        if i == j:
            print("--- Son iguales ---")
            return
        print_topic(topic)
        if not silence:
            confirm = input("confirm")
            if confirm in ["y", "yes", "si"]:
                db_tw.coll.topics.update_one({"query": topic.query}, {"$set": topic.model_dump()})


WITH_SAVE = False
DATE_FROM = datetime(year=2023, month=12, day=28)
DATE_TO = datetime(year=2024, month=1, day=21)
df_accumulated = db_tw.get_trends_df_accumulated(date_from=DATE_FROM, date_to=DATE_TO, with_save=WITH_SAVE)

idx_topic = 9
trend_name = df_accumulated.loc[idx_topic, "name"]
update_topic(
    topic = topic_from_trend(trend_name),
    new_val = "World news",
    old_val = 4,
    silence = False
)
df_accumulated[idx_topic:idx_topic+5][["volume", "name", "topics_1_a", "topics_1_b", "topics_2_a", "topics_2_b"]]

In [ ]:
from typing import List
from scraping_kit.db.db_twitter import DBTwitter
def texts_from_trend(trend_name: str) -> List[str]:
    return [t["text"] for t in db_tw.coll.search.find_one({"query": trend_name})["timeline"]]
texts_from_trend(trend_name)

In [ ]:
fake_doc = db_tw.coll.topics.find_one({"query": "Constitution"})
fake_doc.pop("_id")
fake_doc["query"] = "E. Jean Carroll"
topic = Topic(**fake_doc)
db_tw.coll.topics.insert_one(topic.model_dump())

In [ ]:
#db_tw.coll.search.insert_one({"query": "E. Jean Carroll"})#.delete_one({"query": "E. Jean Carroll"})


In [ ]:
db_tw.coll.topics.find_one({"query": "E. Jean Carroll"})

In [ ]:

texts_from_trend(db_tw, "Feliz 2024")

In [ ]:
db_tw.coll.topics.find_one({"query": "Feliz 2024"})["topics_2"]

In [ ]:
l = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
i, j = 2, 6
l[i], l[j] = l[j], l[i]
l